In [1]:
from os import environ

environ["AWS_ACCESS_KEY_ID"]="dummy"
environ["AWS_SECRET_ACCESS_KEY"]="dummy"
environ["AWS_SESSION_TOKEN"]="dummy"
environ["SEQREPO_ROOT_DIR"]="/usr/local/share/seqrepo/2024-12-20"

import metakb.harvesters.cbioportal as cbph
import metakb.transformers.cbioportal as cbpt
import pandas as pd

from ga4gh.core.models import (
    Coding,
    ConceptMapping,
    Extension,
    MappableConcept,
    Relation,
)

200
Downloaded es_dfarber_broad_2014.tar.gz
Extracted to: es_dfarber_broad_2014_extracted


In [2]:
# Run while inside the harvesters directory
# f = cbph.cBioportalHarvester()
# data = f.harvest()
# g = cbpt.cBioportalTransformer()
# df = g.transform(data)

In [3]:
# df.to_excel('test_file.xlsx')

## MappableConcept
Now with the data from the cBioportal harvester + transformer, we can start writing a loop to get variant data processed

In [4]:
df = pd.read_excel('test_file.xlsx',index_col=0)
df.head()

,Hugo_Symbol,Chromosome,Start_Position,End_Position,Consequence,Variant_Classification,Variant_Type,Reference_Allele,Tumor_Seq_Allele2,SAMPLE_ID,...,temp_Gnomad_Notation,Chrom_23,Chr23_X,Chr23_Y,x_hgnc_id,y_hgnc_id,ambig_chrom,gene_hgnc_id,hgnc_id_match,Gnomad_Notation
0,ETFDH,4,159603535,159603535,missense_variant,Missense_Mutation,SNP,G,C,SJDES004,...,4-159603535-G-C,False,False,False,no_value,no_value,non-ambiguous,untested,untested,4-159603535-G-C
1,GJB4,1,35227334,35227334,missense_variant,Missense_Mutation,SNP,G,A,SJDES004,...,1-35227334-G-A,False,False,False,no_value,no_value,non-ambiguous,untested,untested,1-35227334-G-A
2,MUC5B,11,1272679,1272679,missense_variant,Missense_Mutation,SNP,G,A,SJDES004,...,11-1272679-G-A,False,False,False,no_value,no_value,non-ambiguous,untested,untested,11-1272679-G-A
3,PCDHAC1,5,140307515,140307515,synonymous_variant,Silent,SNP,G,A,SJDES004,...,5-140307515-G-A,False,False,False,no_value,no_value,non-ambiguous,untested,untested,5-140307515-G-A
4,YAP1,11,102080254,102080254,missense_variant,Missense_Mutation,SNP,C,T,SJDES004,...,11-102080254-C-T,False,False,False,no_value,no_value,non-ambiguous,untested,untested,11-102080254-C-T


In [5]:
df["gene_hgnc_id"].value_counts()

gene_hgnc_id
untested    11427
674             2
685             2
6809            2
24358           1
10984           1
11079           1
11123           1
11271           1
11335           1
30681           1
11600           1
24553           1
3051            1
12632           1
12609           1
25117           1
11486           1
12667           1
31838           1
29237           1
12823           1
447             1
29377           1
12976           1
25955           1
10923           1
9959            1
29245           1
4839            1
24009           1
28509           1
1079            1
2435            1
2928            1
20794           1
4283            1
4691            1
32979           1
30892           1
15805           1
11114           1
6808            1
31689           1
11957           1
23485           1
19374           1
22982           1
18743           1
9896            1
21039           1
Name: count, dtype: int64

In [6]:
for key in df.keys():
    print(f'{key}: {df[key][0]}')


Hugo_Symbol: ETFDH
Chromosome: 4
Start_Position: 159603535
End_Position: 159603535
Consequence: missense_variant
Variant_Classification: Missense_Mutation
Variant_Type: SNP
Reference_Allele: G
Tumor_Seq_Allele2: C
SAMPLE_ID: SJDES004
Sequence_Source: WXS
HGVSc: ENST00000511912.1:c.364G>C
HGVSp: p.Gly122Arg
HGVSp_Short: p.G122R
Transcript_ID: ENST00000511912
RefSeq: NM_004453.2
Protein_position: 122
PATIENT_ID: SJDES004
SAMPLE_CLASS: Tumor
ONCOTREE_CODE: ES
CANCER_TYPE: Bone Cancer
CANCER_TYPE_DETAILED: Ewing Sarcoma
TMB_NONSYNONYMOUS: 0.3
AGE: 15
SEX: Male
ETHNICITY: White/Europe
STUDY_ID: es_dfarber_broad_2014
temp_Gnomad_Notation: 4-159603535-G-C
Chrom_23: False
Chr23_X: False
Chr23_Y: False
x_hgnc_id: no_value
y_hgnc_id: no_value
ambig_chrom: non-ambiguous
gene_hgnc_id: untested
hgnc_id_match: untested
Gnomad_Notation: 4-159603535-G-C


In [7]:
import requests
from tqdm import tqdm
from ga4gh.core.models import (
    Coding,
    ConceptMapping,
    Extension,
    MappableConcept,
    Relation,
)

all_mappings = []

base_url = 'https://normalize.cancervariants.org/variation/to_vrs?q=' # 4-159603535-G-C
for idx, row in tqdm(df[0:50].iterrows()):
    variant_of_interest = row['Gnomad_Notation']
    var_url = base_url + variant_of_interest
    r = requests.get(var_url)   
    if r.status_code == 200:
        try:
            vrs_id = r.json()['variations'][0]['id'] 
        except:
            vrs_id = f'Error with {variant_of_interest}'
    else:
        vrs_id = 'Not Found'

    mappings = [
                ConceptMapping(
                    coding=Coding(
                        id=vrs_id,
                        code=str(vrs_id),
                        system="https://www.cbioportal.org/",
                    ),
                    relation=Relation.EXACT_MATCH,
                )
            ]
    
    all_mappings.append(mappings)

# Code to convert all mappings to JSON object goes here

print(all_mappings)



50it [00:06,  7.23it/s]

[[ConceptMapping(id=None, extensions=None, coding=Coding(id='ga4gh:VA.A_KHwjPFBBTw2J2AwTbiEDYA1AZM0-T0', extensions=None, name=None, system='https://www.cbioportal.org/', systemVersion=None, code=code(root='ga4gh:VA.A_KHwjPFBBTw2J2AwTbiEDYA1AZM0-T0'), iris=None), relation='exactMatch')], [ConceptMapping(id=None, extensions=None, coding=Coding(id='ga4gh:VA.lHm5O2TRaxF1tYvxutOofpC40WTjGHea', extensions=None, name=None, system='https://www.cbioportal.org/', systemVersion=None, code=code(root='ga4gh:VA.lHm5O2TRaxF1tYvxutOofpC40WTjGHea'), iris=None), relation='exactMatch')], [ConceptMapping(id=None, extensions=None, coding=Coding(id='ga4gh:VA.F5cXKsHFc7TKvrcysBAzSUIWzve89eoT', extensions=None, name=None, system='https://www.cbioportal.org/', systemVersion=None, code=code(root='ga4gh:VA.F5cXKsHFc7TKvrcysBAzSUIWzve89eoT'), iris=None), relation='exactMatch')], [ConceptMapping(id=None, extensions=None, coding=Coding(id='ga4gh:VA.8mB3IFMdAVHOn0Pv5sZbijYClEXZhJPi', extensions=None, name=None, sys

In [8]:
type(mappings[0])

ga4gh.core.models.ConceptMapping

In [9]:
r.json() # inspect full response if code == 200

{'search_term': '16-3340401-C-A',
 'warnings': [],
 'variations': [{'id': 'ga4gh:VA.ijmmVdDcPm-AvDVr94F9tNJnC61pvkEk',
   'type': 'Allele',
   'digest': 'ijmmVdDcPm-AvDVr94F9tNJnC61pvkEk',
   'location': {'id': 'ga4gh:SL.Wzcz9PAfx1b4LJP_a9B98YB1Wp-K9sxA',
    'type': 'SequenceLocation',
    'digest': 'Wzcz9PAfx1b4LJP_a9B98YB1Wp-K9sxA',
    'sequenceReference': {'type': 'SequenceReference',
     'refgetAccession': 'SQ.W6wLoIFOn4G7cjopxPxYNk2lcEqhLQFb'},
    'start': 3340400,
    'end': 3340401,
    'sequence': 'C'},
   'state': {'type': 'LiteralSequenceExpression', 'sequence': 'A'}}],
 'service_meta_': {'name': 'variation-normalizer',
  'version': '0.13.0',
  'response_datetime': '2025-09-15T22:08:23.484652Z',
  'url': 'https://github.com/cancervariants/variation-normalization'}}

In [10]:
def _get_exact_gene_mappings(hgnc_id, gene_symbol: str) -> list[ConceptMapping]:
        if not hgnc_id and hgnc_id == "untested":
            return None

        gene_id = f"hgnc:{hgnc_id}"

        return[
            ConceptMapping(
                coding=Coding(
                    id= gene_id,
                    name=gene_symbol,
                    code=gene_id.upper(),
                    system="https://www.genenames.org/data/gene-symbol-report/#!/hgnc_id/",
                ),
                relation=Relation.EXACT_MATCH,
                extensions= [Extension(name="cbioportal_annotation", value=True)],
            )]

In [16]:
from typing import List
import logging
from metakb.transformers.base import Transformer
_logger = logging.getLogger(__name__)

class CBPTransformer(Transformer):
    """A """
    def _create_cache(self):
        pass

    def _get_therapeutic_substitute_group(self):
        pass

    def _get_therapy(self):
        pass

    def transform(self):
        pass

    def _add_genes(self, genes:list[dict]) -> list:
        """Create gene objects for all cbioportal records

        """
        transform_genes = []

        for gene in genes:
            gene_symbol = gene.get("Hugo_Symbol")
            hgnc_id = gene.get("gene_hgnc_id")


            queries = [hgnc_id, gene_symbol] if hgnc_id and hgnc_id != "untested" else [gene_symbol]
            extensions = []

            normalized_gene_id = None
            gene_norm_resp = None

            for query in queries:
                gene_norm_resp, normalized_gene_id = self.vicc_normalizers.normalize_gene(query)
                if normalized_gene_id:
                    break

            cbp_mappings = _get_exact_gene_mappings(hgnc_id,gene_symbol)

            if not normalized_gene_id:
                _logger.debug(
                    "Gene Normalizer unable to normalize: using queries %s",
                    queries,
                )
                mappings = cbp_mappings
                extensions.append(self._get_vicc_normalizer_failure_ext())
            else:
                mappings = self._get_vicc_normalizer_mappings(normalized_gene_id, gene_norm_resp)
                #self._update_normalizer_mappings(mappings, _get_exact_gene_mappings) 
                #TODO: add this back, figure out how

            cbp_gene = MappableConcept(
                conceptType="Gene",
                name=gene_symbol,
                mappings=mappings,
                extensions=extensions or None,
            )
            transform_genes.append(cbp_gene)

            #self._cache.genes[gene_symbol] = cbp_gene
        return transform_genes

In [18]:
mappings = _get_exact_gene_mappings("13056","ZNF263")
mappings

[ConceptMapping(id=None, extensions=[Extension(id=None, extensions=None, name='cbioportal_annotation', value=True, description=None)], coding=Coding(id='hgnc:13056', extensions=None, name='ZNF263', system='https://www.genenames.org/data/gene-symbol-report/#!/hgnc_id/', systemVersion=None, code=code(root='HGNC:13056'), iris=None), relation='exactMatch')]

In [13]:
genes = df.to_dict(orient='records')
sub_genes = genes[0:5]

In [14]:
transformer = CBPTransformer()

***Using Gene Database Endpoint: http://localhost:8000***


In [15]:
transformer._add_genes(sub_genes)

[MappableConcept(id=None, extensions=None, conceptType='Gene', name='ETFDH', primaryCoding=None, mappings=[ConceptMapping(id=None, extensions=[Extension(id=None, extensions=None, name='vicc_normalizer_priority', value=True, description=None)], coding=Coding(id='hgnc:3483', extensions=None, name='ETFDH', system='https://www.genenames.org/data/gene-symbol-report/#!/hgnc_id/', systemVersion=None, code=code(root='HGNC:3483'), iris=None), relation='exactMatch'), ConceptMapping(id=None, extensions=[Extension(id=None, extensions=None, name='vicc_normalizer_priority', value=False, description=None)], coding=Coding(id='ncbigene:2110', extensions=None, name=None, system='https://www.ncbi.nlm.nih.gov/gene/', systemVersion=None, code=code(root='2110'), iris=None), relation='exactMatch')]),
 MappableConcept(id=None, extensions=None, conceptType='Gene', name='GJB4', primaryCoding=None, mappings=[ConceptMapping(id=None, extensions=[Extension(id=None, extensions=None, name='vicc_normalizer_priority', 